In [1]:
import requests
import os
import hashlib
import io
import time
import pandas as pd

In [1]:
# Download the db
"https://onedrive.live.com/download?cid=FD520DDC6BE92730&resid=FD520DDC6BE92730%21616&authkey=AEeP_4E1uh-vyDE"

'https://onedrive.live.com/download?cid=FD520DDC6BE92730&resid=FD520DDC6BE92730%21616&authkey=AEeP_4E1uh-vyDE'

In [2]:
# Connect
DB_FILENAME = 'NYC-311-2M.db'
import sqlite3 as db
disk_engine = db.connect(DB_FILENAME)

In [4]:
query = '''
    SELECT ComplaintType, CreatedDate, City
    FROM data
    WHERE CreatedDate >= "2015-09-15 00:00:00.0"
        AND CreatedDate < "2015-09-16 00:00:00.0"
    ORDER BY CreatedDate
    LIMIT 20
'''

df = pd.read_sql_query(query, disk_engine)
df

,ComplaintType,CreatedDate,City
0,Illegal Parking,2015-09-15 00:01:23.000000,None
1,Blocked Driveway,2015-09-15 00:02:29.000000,REGO PARK
2,Taxi Complaint,2015-09-15 00:02:34.000000,NEW YORK
3,Opinion for the Mayor,2015-09-15 00:03:07.000000,None
4,Opinion for the Mayor,2015-09-15 00:03:07.000000,None
5,Noise - Vehicle,2015-09-15 00:03:14.000000,BROOKLYN
6,Dirty Conditions,2015-09-15 00:04:00.000000,BROOKLYN
7,Noise - Commercial,2015-09-15 00:04:22.000000,NEW YORK
8,UNSANITARY CONDITION,2015-09-15 00:04:24.000000,NEW YORK
9,PAINT/PLASTER,2015-09-15 00:04:24.000000,NEW YORK


In [5]:
# This next example shows how to extract just the hour from the time stamp, using SQL's strftime().
query ='''
    SELECT CreatedDate, strftime('%H', CreatedDate) AS Hour, Complainttype
    FROM data
    LIMIT 5
'''

df = pd.read_sql_query(query, disk_engine)
df

,CreatedDate,Hour,ComplaintType
0,2015-09-15 02:14:04.000000,02,Illegal Parking
1,2015-09-15 02:12:49.000000,02,Noise - Street/Sidewalk
2,2015-09-15 02:11:19.000000,02,Noise - Street/Sidewalk
3,2015-09-15 02:09:46.000000,02,Noise - Commercial
4,2015-09-15 02:08:01.000000,02,Homeless Person Assistance


In [6]:
query = '''
  SELECT strftime ('%H', CreatedDate) AS hour, COUNT(*) AS count
    FROM data 
    GROUP BY hour
'''
df_complaints_by_hour = pd.read_sql_query (query, disk_engine)
df_complaints_by_hour

,hour,count
0,00,564703
1,01,23489
2,02,15226
3,03,10164
4,04,8692
5,05,10224
6,06,23051
7,07,42273
8,08,73811
9,09,100077


In [7]:
# An unusual aspect of these data are the excessively large number of reports associated with hour 0 
query = '''
  SELECT COUNT(*)
    FROM data
    WHERE strftime ('%H:%M:%f', CreatedDate) = '00:00:00.000'
'''

pd.read_sql_query(query, disk_engine)

,COUNT(*)
0,532285


In [8]:
disk_engine.close()